In [1]:
import json
import re
import requests
from bs4 import BeautifulSoup
from six import u
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import re
import jieba
import jieba.analyse
from datetime import datetime
from wordcloud import WordCloud
import math
from nltk import ngrams
from collections import defaultdict
from nltk import FreqDist
from collections import Counter, namedtuple
import networkx as nx
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from numpy.linalg import norm
import gensim
from gensim.models import CoherenceModel
from gensim import corpora

In [3]:
with open('detail_articles.json', 'r', encoding='utf-8') as f:
    temp = json.load(f)

In [4]:
cols = ['article_id', 'article_title', 'author', 'board', 'date', 'content', 'message_count', 'push_content',
        'push_userid', 'push_tag']
rows = []
for data in temp:
    data_id = data['article_id']
    message_count = data['message_count']['all']
    messages = data['messages']
    for m in messages:
        rows.append([data_id, data['article_title'], data['author'], data['board'], data['date'], data['content'],
                     message_count, m['push_content'], m['push_userid'], m['push_tag']])

df = pd.DataFrame(rows, columns=cols)

f.close()

In [5]:

df.head(3)


,article_id,article_title,author,board,date,content,message_count,push_content,push_userid,push_tag
0,M.1652148190.A.95B,[問卦] 烏克蘭會將這次軍事應用技術給中國嗎,kogsww (林晨黎),Gossiping,Tue May 10 10:03:08 2022,如蹄， 烏克蘭輸出軍工技術給中國，瓦良格號、氣墊登陸艇、空射巡航導彈、戰鬥機引擎等等...也...,29,現在是美軍的科技你覺得怎麼給= =,thelittleone,噓
1,M.1652148190.A.95B,[問卦] 烏克蘭會將這次軍事應用技術給中國嗎,kogsww (林晨黎),Gossiping,Tue May 10 10:03:08 2022,如蹄， 烏克蘭輸出軍工技術給中國，瓦良格號、氣墊登陸艇、空射巡航導彈、戰鬥機引擎等等...也...,29,不然烏克蘭怎麼賺錢？第一大貿易夥伴欸,zeumax,推
2,M.1652148190.A.95B,[問卦] 烏克蘭會將這次軍事應用技術給中國嗎,kogsww (林晨黎),Gossiping,Tue May 10 10:03:08 2022,如蹄， 烏克蘭輸出軍工技術給中國，瓦良格號、氣墊登陸艇、空射巡航導彈、戰鬥機引擎等等...也...,29,想當下一個伊朗唷,thelittleone,→


In [6]:
with open('detail_articles.json', 'r', encoding='utf-8') as f:
    temp = json.load(f)
cols = ['article_id', 'article_title', 'author', 'board', 'date', 'content', 'sum', 'boo', 'count', 'neutral', 'push']
rows = []


In [7]:
for data in temp:
    message_count = data['message_count']
    sum = data['message_count']['all']
    boo = data['message_count']['boo']
    count = data['message_count']['count']
    neutral = data['message_count']['neutral']
    push = data['message_count']['push']
    rows.append(
        [data['article_id'], data['article_title'], data['author'], data['board'], data['date'], data['content'], sum,
         boo, count, neutral, push])

In [8]:
Meta = pd.DataFrame(rows, columns=cols)


In [9]:
Meta.head(3)

,article_id,article_title,author,board,date,content,sum,boo,count,neutral,push
0,M.1652148190.A.95B,[問卦] 烏克蘭會將這次軍事應用技術給中國嗎,kogsww (林晨黎),Gossiping,Tue May 10 10:03:08 2022,如蹄， 烏克蘭輸出軍工技術給中國，瓦良格號、氣墊登陸艇、空射巡航導彈、戰鬥機引擎等等...也...,29,4,2,19,6
1,M.1652118133.A.C65,[新聞] 普丁震怒！他忙著攻打烏克蘭 美艷情婦卻,"lolahjy (我是貓,名字還沒有)",Gossiping,Tue May 10 01:42:11 2022,https://www.ettoday.net/news/20220509/2247362....,48,4,21,19,25
2,M.1652109371.A.02F,[新聞] 俄烏戰爭若繼續打下去 郭正亮：這國會非,wendsho (wind),Gossiping,Mon May 9 23:16:08 2022,1.媒體來源: 中時新聞網 2.記者署名: 李俊毅 3.完整新聞標題: 俄烏戰爭若繼續打下去...,7,4,-2,1,2


## 資覅清理

In [13]:
Meta.head(5)

,article_id,article_title,author,board,date,content,sum,boo,count,neutral,push
0,M.1652148190.A.95B,[問卦] 烏克蘭會將這次軍事應用技術給中國嗎,kogsww (林晨黎),Gossiping,Tue May 10 10:03:08 2022,如蹄， 烏克蘭輸出軍工技術給中國，瓦良格號、氣墊登陸艇、空射巡航導彈、戰鬥機引擎等等...也...,29,4,2,19,6
1,M.1652118133.A.C65,[新聞] 普丁震怒！他忙著攻打烏克蘭 美艷情婦卻,"lolahjy (我是貓,名字還沒有)",Gossiping,Tue May 10 01:42:11 2022,,48,4,21,19,25
2,M.1652109371.A.02F,[新聞] 俄烏戰爭若繼續打下去 郭正亮：這國會非,wendsho (wind),Gossiping,Mon May 9 23:16:08 2022,1.: 中時新聞網 2.: 李俊毅 3.: 俄烏戰爭若繼續打下去 郭正亮：這國會非常緊張 4...,7,4,-2,1,2
3,M.1652106647.A.242,[問卦] 5/9 俄羅斯？,waymayday (不要問),Gossiping,Mon May 9 22:30:45 2022,五月九號 最後通令 俄羅斯 布丁大帝不是說要把烏克蘭打下來 結果現在？ 各國政要跑去基輔泡茶...,8,1,2,4,3
4,M.1652105951.A.33C,[問卦] 為何烏克蘭不攻打俄羅斯？,dubliuers (豆漿),Gossiping,Mon May 9 22:19:09 2022,為什麼烏克蘭只能一直防禦 不直接射一顆飛彈到克林姆林宮？ 讓普丁投降？ 一直防禦是因為怕痛嗎...,10,0,0,10,0


In [11]:
Meta['content'] = Meta.content.apply(
    lambda x: re.sub('(媒體來源|記者署名|完整新聞標題|完整新聞內文|完整新聞連結|(或短網址)|備註:|備註請放最後面|違者新聞文章刪除)', '', str(x)))
Meta['content'] = Meta.content.apply(lambda x: re.sub('(http|https)://.*', '', x))
Meta['content'] = Meta.content.apply(lambda x: re.sub('\n', ',', x))
Meta['content'] = Meta.content.apply(lambda x: re.sub('\n', ',', x))

In [14]:
Meta.head(5)


,article_id,article_title,author,board,date,content,sum,boo,count,neutral,push
0,M.1652148190.A.95B,[問卦] 烏克蘭會將這次軍事應用技術給中國嗎,kogsww (林晨黎),Gossiping,Tue May 10 10:03:08 2022,如蹄， 烏克蘭輸出軍工技術給中國，瓦良格號、氣墊登陸艇、空射巡航導彈、戰鬥機引擎等等...也...,29,4,2,19,6
1,M.1652118133.A.C65,[新聞] 普丁震怒！他忙著攻打烏克蘭 美艷情婦卻,"lolahjy (我是貓,名字還沒有)",Gossiping,Tue May 10 01:42:11 2022,,48,4,21,19,25
2,M.1652109371.A.02F,[新聞] 俄烏戰爭若繼續打下去 郭正亮：這國會非,wendsho (wind),Gossiping,Mon May 9 23:16:08 2022,1.: 中時新聞網 2.: 李俊毅 3.: 俄烏戰爭若繼續打下去 郭正亮：這國會非常緊張 4...,7,4,-2,1,2
3,M.1652106647.A.242,[問卦] 5/9 俄羅斯？,waymayday (不要問),Gossiping,Mon May 9 22:30:45 2022,五月九號 最後通令 俄羅斯 布丁大帝不是說要把烏克蘭打下來 結果現在？ 各國政要跑去基輔泡茶...,8,1,2,4,3
4,M.1652105951.A.33C,[問卦] 為何烏克蘭不攻打俄羅斯？,dubliuers (豆漿),Gossiping,Mon May 9 22:19:09 2022,為什麼烏克蘭只能一直防禦 不直接射一顆飛彈到克林姆林宮？ 讓普丁投降？ 一直防禦是因為怕痛嗎...,10,0,0,10,0


In [15]:
jieba.set_dictionary('G:\我的雲端硬碟\社群媒體探勘\HW\Project\code\dataset\jieba\dict.txt.big')
with open('G:\我的雲端硬碟\社群媒體探勘\HW\Project\code\dataset\jieba\stop_words.txt', encoding="utf-8") as f:
    stopWords = [line.strip() for line in f.readlines()]

In [16]:
def getToken(row):
    seg_list = jieba.lcut(row)
    seg_list = [w for w in seg_list if w not in stopWords and len(w) > 1]  # 篩選不在停用字的字與字元數大於1的字詞
    return seg_list

In [17]:
data = Meta.copy()
data['word']=data.content.apply(getToken)


AttributeError: module 'jieba' has no attribute 'lcut'